**Simple** model
Based on deep learning course

with use transfer learning

In [0]:
! pip install pytorch-pretrained-bert

In [0]:
# for colab 

# 0. imports 

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Mount
from google.colab import drive
drive.mount('/content/drive')

In [3]:
% cd /content/drive/'My Drive'/master_thesis
% ls 

/content/drive/My Drive/master_thesis
bert_model_test.pth  convert_examples_to_features.py  outputs/      reports/
cache/               data/                            __pycache__/  tools.py


In [4]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [0]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [0]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from random import randrange
import torch.nn.functional as F
from torch.nn import RNN

In [0]:
from pytorch_pretrained_bert import BertConfig

config = BertConfig(vocab_size_or_config_json_file=32000, hidden_size=768,
        num_hidden_layers=12, num_attention_heads=12, intermediate_size=3072)

num_labels = 2
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

In [0]:
#dat = pd.read_csv('data/final_full.csv')
dat = pd.read_csv('data/final_sub.csv')
dat.head()

X = dat['reviewText']
y = dat['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [0]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()

y_train = pd.get_dummies(y_train).values.tolist()
y_test = pd.get_dummies(y_test).values.tolist()

In [0]:
max_seq_length = 256
# to check 

class text_dataset(Dataset):
    def __init__(self,x_y_list, transform=None):
        
        self.x_y_list = x_y_list
        self.transform = transform
        
    def __getitem__(self,index):
        
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index])
        
        if len(tokenized_review) > max_seq_length:
            tokenized_review = tokenized_review[:max_seq_length]
            
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)

        padding = [0] * (max_seq_length - len(ids_review))
        
        ids_review += padding
        
        assert len(ids_review) == max_seq_length
        
        #print(ids_review)
        ids_review = torch.tensor(ids_review)
        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        
        
        return ids_review, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

In [0]:
class BertForSequenceClassification(nn.Module):
    
    def __init__(self, num_labels=2, hidden_size=128):
        super(BertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(10, num_labels)
        nn.init.xavier_normal_(self.classifier.weight)
        self.rnn = nn.RNN(config.hidden_size, hidden_size, 10)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.rnn
        logits = self.classifier(pooled_output)

        return logits

In [12]:
batch_size = 16

train_lists = [X_train, y_train]
test_lists = [X_test, y_test]

training_dataset = text_dataset(x_y_list = train_lists )

test_dataset = text_dataset(x_y_list = test_lists )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
def train_model(model, loss_fn, optimizer, num_epochs=25):
    since = time.time()
    print('starting')
    
    phase = 'train'
    model.train()
    
    loss_train = np.zeros(num_epochs)
    acc_train = np.zeros(num_epochs)
    

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)



        running_loss = 0.0
        running_corrects = 0


        for inputs, labels in dataloaders_dict[phase]:
          
            inputs = inputs.to(device) 
            labels = labels.to(device)
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
              
              
              outputs = model(inputs)
              outputs = F.softmax(outputs,dim=1)
              loss = loss_fn(outputs, torch.max(labels.float(), 1)[1])

              loss.backward()
              optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(labels, 1)[1])
           

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        loss_train[epoch] = epoch_loss
        acc_train[epoch] = epoch_acc
        
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    return loss_train, acc_train

In [0]:
model.to(device)

for param in model.bert.parameters():
            param.requires_grad = True
    
#model.classifier.weight.requires_grad = True

#model

In [0]:
lrlast = .001
lrmain = .00001

optim1 = optim.Adam(model.parameters(), lr=lrmain)
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

In [16]:
model_ft1 = train_model(model, criterion, optimizer_ft, num_epochs=5)

starting
Epoch 0/4
----------
Train - Loss: 0.5610 Acc: 0.7291
Epoch 1/4
----------
Train - Loss: 0.3628 Acc: 0.9529
Epoch 2/4
----------
Train - Loss: 0.3437 Acc: 0.9704
Epoch 3/4
----------
Train - Loss: 0.3445 Acc: 0.9696
Epoch 4/4
----------
Train - Loss: 0.3380 Acc: 0.9758
Training complete in 21m 43s


In [0]:
def test(model, loss_fn):
    
    phase = 'val'
    model.eval()

    running_corrects = 0.0
    running_loss = 0.0
    

    for inputs, labels in dataloaders_dict[phase]:

      inputs = inputs.to(device) 
      labels = labels.to(device)



      outputs = model(inputs)
      loss = loss_fn(outputs, torch.max(labels.float(), 1)[1])


      # statistics
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(labels, 1)[1])
   
    val_loss = running_loss / dataset_sizes[phase]
    val_acc = running_corrects.double() / dataset_sizes[phase]  

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))

In [18]:
test(model, criterion)

Test - Loss: 0.3285 Acc: 0.9400
